In [1]:
import os
import pandas as pd
import sys

os.chdir("/root/data2/ghc/RAG/lin_rany/code/dl_learn/transform")
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 


sys.argv = ['myscript.py', 'arg1', 'arg2']

/root/data2/ghc/RAG/lin_rany/code/dl_learn/transform


In [2]:
from torch.utils.data import Dataset
class MyDataSet(Dataset):
    def __init__(self, data_file):
        self.data = self.load_data(data_file)
    def load_data(self, data_file):
        
        data=pd.read_csv(data_file,sep='\t')
        # print(data.head())
        
        Data=list_of_dict = data.to_dict('records')
        numpy_index = data.index.to_numpy()
        numpy_columns = data.columns.to_numpy()
        print(numpy_index)
        print(numpy_columns)
        return Data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_data = MyDataSet('datasets/x-final/zh/translated_train.tsv')
valid_data = MyDataSet('datasets/x-final/zh/dev_2k.tsv')

print(train_data[:3])
print(train_data.__len__())
print(f"finish load data")

[    0     1     2 ... 49398 49399 49400]
['id' 'sentence1' 'sentence2' 'label']
[   0    1    2 ... 1997 1998 1999]
['id' 'sentence1' 'sentence2' 'label']
[{'id': 1, 'sentence1': '1560年10月，他在巴黎秘密会见了英国大使Nicolas Throckmorton，要求他通过苏格兰返回英国。', 'sentence2': '1560年10月，他在巴黎秘密会见了英国大使尼古拉斯·斯罗克莫顿，并要求他通过英格兰返回苏格兰的护照。', 'label': 0}, {'id': 2, 'sentence1': '1975年的NBA赛季 -  76赛季是全美篮球协会的第30个赛季。', 'sentence2': '1975-76赛季的全国篮球协会是NBA的第30个赛季。', 'label': 1}, {'id': 3, 'sentence1': '还有具体的讨论，公众形象辩论和项目讨论。', 'sentence2': '还有公开讨论，特定档案讨论和项目讨论。', 'label': 0}]
49401
finish load data


In [3]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

checkpoint = "bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/data/zhongyuan_peng/anaconda3/envs/llm_lin_rany/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def collate_fn(examples):
    batch_sentence_1, batch_sentence_2 = [], []
    batch_label = []
    for sample in examples:
        batch_sentence_1.append(str(sample['sentence1']))
        batch_sentence_2.append(str(sample['sentence2']))
        batch_label.append(int(sample['label']))
    ret=tokenizer(
        batch_sentence_1, 
        batch_sentence_2, 
        padding=True, 
        truncation=True, 
        return_tensors="pt"
    )
    y = torch.tensor(batch_label)
    return ret,y
train_dataloader=DataLoader(train_data,batch_size=64,shuffle=True,collate_fn=collate_fn)
x,y=next(iter(train_dataloader))
x_map={k:v.shape for k, v in x.items()}
print(f"x_map: {x_map} y: {y}")

x_map: {'input_ids': torch.Size([64, 124]), 'token_type_ids': torch.Size([64, 124]), 'attention_mask': torch.Size([64, 124])} y: tensor([0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0])


In [5]:
from torch import nn
from transformers import AutoConfig
from transformers import BertPreTrainedModel, BertModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [6]:
class MyModelForBertForPairwiseCLS(BertPreTrainedModel):
    def __init__(self,config):
        super().__init__(config)
        self.bert = BertModel(config,add_pooling_layer=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(768,2)
        self.post_init()
    def forward(self, input):
        bert_out=self.bert(**input)
        cls_vertors=bert_out.last_hidden_state[:,0,:]
        cls_vertors=self.dropout(cls_vertors)
        logits=self.classifier(cls_vertors)
        return logits
config = AutoConfig.from_pretrained(checkpoint)
model = MyModelForBertForPairwiseCLS.from_pretrained(checkpoint,config=config).to(device)
print(model)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

MyModelForBertForPairwiseCLS(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-

In [7]:
x=x.to(device)
x_out=model(x)
print(f"x_out: {x_out.shape}")

x_out: torch.Size([64, 2])


In [8]:
from tqdm.auto import tqdm



def train_loop(dataloader,model,loss_fn,optimizer,lr_scheduler,epoch,total_loss):
    progress_bar = tqdm(range(len(dataloader)))
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_step_num = (epoch-1)*len(dataloader)
    model.train()
    
    for step, (X, y) in enumerate(dataloader, start=1):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        progress_bar.set_description(f'loss: {total_loss/(finish_step_num + step):>7f}')
        progress_bar.update(1)
    return total_loss
def test_loop(dataloader, model, mode='Test'):
    assert mode in ['Valid', 'Test']
    size = len(dataloader.dataset)
    correct = 0

    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    correct /= size
    print(f"{mode} Accuracy: {(100*correct):>0.1f}%\n")




In [9]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-3
epoch_num = 3

loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=learning_rate)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    test_loop(valid_dataloader, model, mode='Valid')
print("Done!")

/data/zhongyuan_peng/anaconda3/envs/llm_lin_rany/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
-------------------------------


loss: 0.631690:  57%|█████▋    | 438/772 [02:01<01:37,  3.42it/s]

KeyboardInterrupt: 